In [3]:
# Importera nödvändiga bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import joblib

# Ladda data
df = pd.read_csv('cardio_train.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../cardio_train.csv'

In [ ]:
# 1.1 EDA (Exploratory Data Analysis)

# a) Hur många är positiva för hjärt-kärlsjukdom och hur många är negativa?
disease_counts = df['cardio'].value_counts()
plt.figure(figsize=(10, 6))
sns.barplot(x=disease_counts.index, y=disease_counts.values)
plt.title('Fördelning av hjärt-kärlsjukdom')
plt.show()

print(f"Antal positiva fall: {disease_counts[1]}")
print(f"Antal negativa fall: {disease_counts[0]}")
print(f"Andel positiva fall: {(disease_counts[1]/len(df))*100:.2f}%")

In [ ]:

# b) Kolesterolvärden
cholesterol_dist = df['cholesterol'].value_counts()
plt.figure(figsize=(10, 6))
sns.barplot(x=cholesterol_dist.index, y=cholesterol_dist.values)
plt.title('Fördelning av kolesterolvärden')
plt.xlabel('1: normal, 2: över normal, 3: långt över normal')
plt.show()

for i in range(1, 4):
    print(f"Andel med kolesterolvärde {i}: {(cholesterol_dist[i]/len(df))*100:.2f}%")


In [ ]:

# c) Åldersfördelning
plt.figure(figsize=(10, 6))
sns.histplot(df['age']/365, bins=30)  # Konvertera dagar till år
plt.title('Åldersfördelning')
plt.xlabel('Ålder (år)')
plt.show()

print(f"Medelålder: {(df['age']/365).mean():.1f} år")
print(f"Medianålder: {(df['age']/365).median():.1f} år")

In [ ]:

# d) Andel som röker
smoking_dist = df['smoke'].value_counts()
print(f"Andel som röker: {(smoking_dist[1]/len(df))*100:.2f}%")


In [ ]:

# e) Viktfördelning
plt.figure(figsize=(10, 6))
sns.histplot(df['weight'], bins=30)
plt.title('Viktfördelning')
plt.xlabel('Vikt (kg)')
plt.show()

print(f"Medelvikt: {df['weight'].mean():.1f} kg")
print(f"Medianvikt: {df['weight'].median():.1f} kg")


In [ ]:

# f) Längdfördelning
plt.figure(figsize=(10, 6))
sns.histplot(df['height'], bins=30)
plt.title('Längdfördelning')
plt.xlabel('Längd (cm)')
plt.show()

print(f"Medellängd: {df['height'].mean():.1f} cm")
print(f"Medianlängd: {df['height'].median():.1f} cm")


In [ ]:
# g) Andel hjärt-kärlsjukdom per kön
gender_disease = pd.crosstab(df['gender'], df['cardio'], normalize='index') * 100
gender_disease.plot(kind='bar', figsize=(10, 6))
plt.title('Andel hjärt-kärlsjukdom per kön')
plt.xlabel('Kön (1: kvinna, 2: man)')
plt.ylabel('Procent')
plt.show()

print("Andel hjärt-kärlsjukdom per kön:")
print(gender_disease)


In [ ]:
# 1.2 Modelldesign - Korrelationsmatris
correlation_matrix = df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Korrelationsmatris')
plt.show()

In [ ]:

# 1.2.1 Feature Engineering

# Beräkna BMI
df['BMI'] = df['weight'] / ((df['height']/100) ** 2)

# Ta bort orimliga BMI-värden (under 15 eller över 50)
df = df[(df['BMI'] >= 15) & (df['BMI'] <= 50)]

# Skapa BMI-kategorier
def get_bmi_category(bmi):
    if bmi < 18.5:
        return 'underweight'
    elif bmi < 25:
        return 'normal'
    elif bmi < 30:
        return 'overweight'
    elif bmi < 35:
        return 'obese_1'
    elif bmi < 40:
        return 'obese_2'
    else:
        return 'obese_3'

df['BMI_category'] = df['BMI'].apply(get_bmi_category)

# Ta bort orimliga blodtrycksvärden
df = df[(df['ap_hi'] > 0) & (df['ap_hi'] < 300) & 
        (df['ap_lo'] > 0) & (df['ap_lo'] < 200) &
        (df['ap_hi'] > df['ap_lo'])]

# Skapa blodtryckskategorier
def get_bp_category(systolic, diastolic):
    if systolic < 120 and diastolic < 80:
        return 'normal'
    elif systolic < 130 and diastolic < 80:
        return 'elevated'
    elif systolic < 140 or diastolic < 90:
        return 'hypertension_1'
    else:
        return 'hypertension_2'

df['BP_category'] = df.apply(lambda x: get_bp_category(x['ap_hi'], x['ap_lo']), axis=1)


In [ ]:

# 1.2.2 Skapa två dataset
# Dataset 1 med kategoriska variabler
df1 = df.copy()
categorical_columns = ['BMI_category', 'BP_category', 'gender']
df1 = pd.get_dummies(df1, columns=categorical_columns)
df1.drop(['ap_hi', 'ap_lo', 'height', 'weight', 'BMI'], axis=1, inplace=True)

# Dataset 2 med numeriska variabler
df2 = df.copy()
df2 = pd.get_dummies(df2, columns=['gender'])
df2.drop(['BMI_category', 'BP_category', 'height', 'weight'], axis=1, inplace=True)


In [ ]:

# 1.2.3 Träning och utvärdering av modeller
from sklearn.model_selection import train_test_split

# Dela upp data
X1 = df1.drop('cardio', axis=1)
y1 = df1['cardio']
X2 = df2.drop('cardio', axis=1)
y2 = df2['cardio']

# Skala data
scaler = StandardScaler()
X1_scaled = scaler.fit_transform(X1)
X2_scaled = scaler.fit_transform(X2)

# Definiera modeller och parametrar
models = {
    'rf': (RandomForestClassifier(), {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None]
    }),
    'lr': (LogisticRegression(), {
        'C': [0.1, 1.0, 10.0],
        'penalty': ['l2']
    }),
    'svm': (SVC(probability=True), {
        'C': [0.1, 1.0, 10.0],
        'kernel': ['rbf', 'linear']
    })
}

In [ ]:


# Träna och utvärdera modeller på båda dataseten
results = {}
for name, (model, params) in models.items():
    # Dataset 1
    grid1 = GridSearchCV(model, params, cv=5, scoring='accuracy')
    grid1.fit(X1_scaled, y1)
    
    # Dataset 2
    grid2 = GridSearchCV(model, params, cv=5, scoring='accuracy')
    grid2.fit(X2_scaled, y2)
    
    results[name] = {
        'dataset1': {'score': grid1.best_score_, 'params': grid1.best_params_},
        'dataset2': {'score': grid2.best_score_, 'params': grid2.best_params_}
    }

# Visa resultat
for model_name, result in results.items():
    print(f"\nModell: {model_name}")
    print(f"Dataset 1 - Score: {result['dataset1']['score']:.4f}")
    print(f"Dataset 1 - Bästa parametrar: {result['dataset1']['params']}")
    print(f"Dataset 2 - Score: {result['dataset2']['score']:.4f}")
    print(f"Dataset 2 - Bästa parametrar: {result['dataset2']['params']}")


In [ ]:

# 1.3 Paketering och demo
# Välj bästa modell och dataset baserat på resultaten
# Antag att RandomForest på dataset 1 var bäst

# Ta ut 100 slumpmässiga rader för test
test_indices = np.random.choice(len(df1), 100, replace=False)
test_samples = df1.iloc[test_indices]
train_samples = df1.drop(test_indices)

# Spara test samples
test_samples.to_csv('test_samples.csv', index=False)

# Träna final modell
best_model = RandomForestClassifier(**results['rf']['dataset1']['params'])
X_train = train_samples.drop('cardio', axis=1)
y_train = train_samples['cardio']
best_model.fit(X_train, y_train)

# Spara modellen
joblib.dump(best_model, 'heart_disease_model.pkl')